### 데이터 다운로드

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

--2024-03-21 03:33:55--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.162.207, 74.125.134.207, 74.125.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.162.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   129MB/s    in 0.5s    

2024-03-21 03:33:56 (129 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
!unzip -qq cats_and_dogs_filtered.zip

In [ ]:
ls

In [ ]:
# 패키지 임포트
import os
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image

os 모듈은 파일 시스템에서 파일을 생성, 삭제, 수정하거나, 파일 경로를 조작하고, 환경 변수에 접근하는 등의 작업 수행

In [ ]:
#pytorch 버전 확인
import torch
print(torch.__version__)

2.2.1+cu121


In [ ]:
# GPU 사용 체크
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True

### 파이토치 데이터셋 클래스

In [ ]:
# 데이터세트 정의 클래스
class PyTorchCustomDataset(Dataset):
    def __init__(self
                 , root_dir = "/content/cats_and_dogs_filtered/train"
                 , transform = None):
        self.image_abs_path = root_dir
        self.transform = transform
        self.label_list = os.listdir(self.image_abs_path)
        self.label_list.sort()
        self.x_list = []
        self.y_list = []
        for label_index, label_str in enumerate(self.label_list):
            img_path = os.path.join(self.image_abs_path, label_str)
            img_list = os.listdir(img_path)
            for img in img_list:
                self.x_list.append(os.path.join(img_path, img))
                self.y_list.append(label_index)
        pass

    def __len__(self):
        return len(self.x_list)

    def __getitem__(self, idx):
        image = Image.open(self.x_list[idx])
        if image.mode is not "RGB":
            image = image.convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, self.y_list[idx]

    def __save_label_map__(self, dst_text_path = "label_map.txt"):
        label_list = self.label_list
        f = open(dst_text_path, 'w')
        for i in range(len(label_list)):
            f.write(label_list[i]+'\n')
        f.close()
        pass

    def __num_classes__(self):
        return len(self.label_list)

<>:25: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:25: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-7-6491a468c92d>:25: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if image.mode is not "RGB":


__init__ 메서드는 클래스의 인스턴스가 생성될 때 초기화를 담당

root_dir 매개변수는 이미지 파일이 위치한 기본 경로를 설정

transform 매개변수는 이미지에 적용할 전처리(transform)를 정의

self.image_abs_path는 이미지 파일의 절대 경로를 저장

self.label_list는 root_dir 경로에 있는 각 레이블(디렉토리)의 목록을 저장

self.x_list와 self.y_list는 각각 이미지 파일의 경로와 해당 이미지의 레이블 인덱스를 저장

모든 레이블에 대해 이미지 경로와 레이블 인덱스를 self.x_list와 self.y_list에 추가

__len__ 메서드는 데이터셋의 총 아이템 수를 반환. 이 경우 self.x_list의 길이를 반환

__getitem__ 메서드는 주어진 인덱스(idx)에 해당하는 아이템(이미지와 레이블)을 데이터셋에서 가져온다.

이미지 파일은 Image.open을 사용하여 열리며, 이미지가 RGB 모드가 아닌 경우 RGB로 변환

설정된 transform이 있다면 이미지에 적용

변환된 이미지와 해당 레이블 인덱스가 반환

__save_label_map__ 메서드는 레이블 목록을 텍스트 파일로 저장

파일 경로는 dst_text_path 매개변수를 통해 지정

각 레이블을 줄 단위로 파일에 쓰고 파일을 닫는다.

__num_classes__ 메서드는 데이터셋의 클래스(레이블) 수를 반환, self.label_list의 길이와 동일

### model

In [ ]:
# 네트워크 정의
import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

class MODEL(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.network = models.resnet18(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Dropout()
            , nn.Linear(1000, num_classes)
            , nn.Sigmoid()
        )
    def forward(self, x):
        x = self.network(x)
        return self.classifier(x)

사전 학습된 ResNet-18 아키텍처를 기반

self.network는 사전 학습된 ResNet-18 모델을 로드

pretrained=True 인자는 사전 학습된 가중치를 사용하여 모델을 초기화

self.classifier는 신경망의 마지막 부분을 사용자 정의 분류기로 대체

nn.Dropout(): 과적합을 방지하기 위해 드롭아웃을 적용. 드롭아웃 비율은 기본값(0.5)을 사용

nn.Linear(1000, num_classes): 선형 레이어(완전 연결 레이어)로, ResNet-18의 마지막 레이어 출력(기본적으로 1000차원)을 num_classes 차원으로 변환

nn.Sigmoid(): 이진 분류 문제를 위한 활성화 함수로 시그모이드 함수, 다중 레이블 분류 문제에 적합하며, 각 클래스에 대한 독립적인 확률을 출력

forward 메서드는 모델이 입력 데이터 x를 받아 출력을 계산하는 방식

### main 함수

In [ ]:
# 훈련 메인 함수 정의
import torch
import torch.optim as optim

train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

def trainmain():
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
    img_width, img_height = 224, 224
    EPOCHS     = 12
    BATCH_SIZE = 32

    #데이터세트 로딩
    transform_train = transforms.Compose([
                transforms.Resize(size=(img_width, img_height))
                , transforms.RandomRotation(degrees=15)
                , transforms.ToTensor()
                ])
    transform_test = transforms.Compose([
                transforms.Resize(size=(img_width, img_height))
                , transforms.ToTensor()
                ])
    TrainDataset = PyTorchCustomDataset
    TestDataset = PyTorchCustomDataset
    train_data = TrainDataset(root_dir = "/content/cats_and_dogs_filtered/train"
                    , transform = transform_train)
    test_data = TestDataset(root_dir = "/content/cats_and_dogs_filtered/validation"
                    , transform = transform_test)
    train_loader = torch.utils.data.DataLoader(
        train_data
        , batch_size=BATCH_SIZE
        , shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_data
        , batch_size=BATCH_SIZE
        , shuffle=True
    )
    train_data.__save_label_map__()
    num_classes = train_data.__num_classes__()

    #모델 객체 생성, PyTorch_Classification_Model.pt 모델 파일명 지정
    model = MODEL(num_classes).to(DEVICE)
    model_str = "PyTorch_Classification_Model"
    model_str += ".pt"

    #최적화 함수와 학습률 지정
    #optimizer = optim.Adam(model.parameters(), lr=0.0001)
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    acc = 0.0

    # 에포크 만큼 훈련, 검증
    for epoch in range(1, EPOCHS + 1):
        model.train()
        tr_loss = 0.0
        tr_correct = 0.0
        for data, target in (train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = F.cross_entropy(output, target)
            tr_loss += F.nll_loss(output,target,reduction='sum').item()
            pred = output.data.max(dim=1,keepdim=True)[1]
            tr_correct += pred.eq(target.view_as(pred)).sum().item()
            loss.backward()
            optimizer.step()
        scheduler.step()
        tr_ep_loss = tr_loss/len(train_loader.dataset)
        tr_ep_accuracy = 100. * tr_correct/len(train_loader.dataset)

        model.eval()
        te_loss = 0
        te_correct = 0
        with torch.no_grad():
            for data, target in (test_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                loss = F.cross_entropy(output, target)
                te_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                te_correct += pred.eq(target.view_as(pred)).sum().item()
        te_ep_loss = te_loss / len(test_loader.dataset)
        te_ep_accuracy = 100. * te_correct / len(test_loader.dataset)
        print('[{}] Train Loss: {:.4f}, Train Accuracy: {:.2f}% Test Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(
                epoch, tr_ep_loss, tr_ep_accuracy, te_ep_loss, te_ep_accuracy))

        if acc < te_ep_accuracy:
            acc = te_ep_accuracy
            torch.save(model.state_dict(), model_str)
            print("model saved!")

        train_losses.append(tr_ep_loss)
        train_accuracy.append(tr_ep_accuracy)
        val_losses.append(te_ep_loss)
        val_accuracy.append(te_ep_accuracy)

이미지의 너비와 높이를 224로 설정하고, 한 번에 처리할 이미지 수인 배치 크기를 32로 설정

훈련 데이터에는 크기 조정, 무작위 회전, Tensor 변환을 적용하고, 테스트 데이터에는 크기 조정과 Tensor 변환을 적용

데이터 로더는 배치 크기에 맞게 데이터를 나누고, 훈련 데이터의 경우 무작위로 섞는다.

Adam 최적화 함수를 사용하고, 학습률 스케줄러를 설정하여 훈련 과정에서 학습률을 조정

각 에포크마다 훈련 및 검증 데이터셋에 대한 손실(loss)과 정확도(accuracy)를 계산

역전파를 통해 모델의 가중치를 업데이트

기울기 계산이 필요 없으므로 torch.no_grad()를 사용

In [ ]:
# 훈련 메인 함수 호출
trainmain()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 108MB/s]


[1] Train Loss: -0.9026, Train Accuracy: 93.95% Test Loss: 0.3490, Test Accuracy: 96.50%
model saved!
[2] Train Loss: -0.9747, Train Accuracy: 98.15% Test Loss: 0.3336, Test Accuracy: 98.10%
model saved!
[3] Train Loss: -0.9823, Train Accuracy: 98.65% Test Loss: 0.3343, Test Accuracy: 97.90%
[4] Train Loss: -0.9876, Train Accuracy: 99.15% Test Loss: 0.3374, Test Accuracy: 97.40%
[5] Train Loss: -0.9860, Train Accuracy: 98.75% Test Loss: 0.3382, Test Accuracy: 97.20%
[6] Train Loss: -0.9926, Train Accuracy: 99.35% Test Loss: 0.3333, Test Accuracy: 98.00%
[7] Train Loss: -0.9929, Train Accuracy: 99.50% Test Loss: 0.3336, Test Accuracy: 97.90%


실제로 이미지 분류 모델의 훈련 및 검증 과정이 시작

train epoch 2까지만 돌고 훈련 진행이 안됨...ㅠ

In [ ]:
# 훈련 데이터와 검증 데이터의 손실 그래프
import matplotlib.pyplot as plt

plt.plot(range(1,len(train_losses)+1),train_losses,'bo',label = 'training loss')
plt.plot(range(1,len(val_losses)+1),val_losses,'r',label = 'validation loss')
plt.legend()

In [ ]:
# 훈련 데이터와 검증 데이터의 정확도 그래프
plt.plot(range(1,len(train_accuracy)+1),train_accuracy,'bo',label = 'train accuracy')
plt.plot(range(1,len(val_accuracy)+1),val_accuracy,'r',label = 'val accuracy')
plt.legend()

In [ ]:
# PyTorch_Classification_Model.pt 모델 파일 확인
!ls

### 이미지 분류 모델 추론 테스트

In [ ]:
# 테스트 이미지 로딩
import os
PATH = "/content/cats_and_dogs_filtered/validation"
validation_cats_dir = PATH + '/cats'  # directory with our validation cat pictures
validation_dogs_dir = PATH + '/dogs'  # directory with our validation dog pictures
list_of_test_cats_images = os.listdir(validation_cats_dir)
list_of_test_dogs_images = os.listdir(validation_dogs_dir)
for idx in range(len(list_of_test_cats_images)):
    list_of_test_cats_images[idx] = validation_cats_dir + '/'+list_of_test_cats_images[idx]
for idx in range(len(list_of_test_dogs_images)):
    list_of_test_dogs_images[idx] = validation_dogs_dir + '/'+list_of_test_dogs_images[idx]
list_of_test_images = list_of_test_cats_images + list_of_test_dogs_images

특정 디렉토리(여기서는 고양이와 개 사진이 있는 검증 데이터셋 디렉토리) 내의 이미지 파일 목록을 생성하고, 각 파일의 전체 경로를 리스트에 저장하는 과정

os.listdir 함수를 사용하여 validation_cats_dir와 validation_dogs_dir 디렉토리 내의 모든 파일(이 경우 이미지) 목록을 각각 가져온다.

각 파일명 앞에 해당 파일의 디렉토리 경로를 추가함으로써 전체 파일 경로로 업데이트. 이는 파일을 접근할 때 필요한 전체 경로를 형성

고양이와 개 이미지 파일의 전체 경로를 포함하는 두 리스트를 합쳐서 list_of_test_images라는 하나의 리스트를 생성

In [ ]:
# 로딩된 이미지 경로 프린트
print(list_of_test_cats_images[10])
print(list_of_test_images[501])

먼저 고양이 이미지 파일 목록이 추가되고 그 다음에 개 이미지 파일 목록이 추가되므로, 501번째 인덱스에 있는 파일이 고양이인지 개인지는 고양이 이미지와 개 이미지의 전체 수에 따라 달라진다.

첫번째가 고양이 그림, 두번째가 강아지 그림이 나온다.

In [ ]:
# 이미지 보여주는 함수, 이미지 추론 함수

from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

#라벨맵 로딩 함수
def load_label_map(textFile):
    return np.loadtxt(textFile, str, delimiter='\t')

#이미지 읽는 함수
def cv_image_read(image_path):
    print(image_path)
    return cv2.imread(image_path)

#이미지 보여주는 함수
def show_image(cv_image):
    rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(rgb)
    plt.show()

#이미지를 분류 모델로 추론한 결과를 텍스트로 보여주는 함수
def print_result(inference_result, class_map):
    class_text = class_map[np.argmax(inference_result)]
    print(inference_result)
    print(class_text)

#이미지를 분류 모델로 추론하는 함수
def inference_image(opencv_image, transform_info, model, DEVICE):
    image = Image.fromarray(opencv_image)
    image_tensor = transform_info(image)
    image_tensor = image_tensor.unsqueeze(0)
    image_tensor = image_tensor.to(DEVICE)
    result = model(image_tensor)
    return result

In [ ]:
# 이미지 보여주는 함수 실습
show_image(cv_image_read(list_of_test_images[10]))
show_image(cv_image_read(list_of_test_images[501]))

In [ ]:
# 테스트 메인 함수
import os
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

def testmain(image_path):
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

    img_width, img_height = 224, 224
    transform_info = transforms.Compose([
                transforms.Resize(size=(img_width, img_height))
                , transforms.ToTensor()
                    ])
    #라벨 파일 읽기
    class_map = load_label_map('label_map.txt')
    num_classes = len(class_map)

    #지정된 모델 로딩
    model = MODEL(num_classes).to(DEVICE)
    model_str = "PyTorch_Classification_Model"
    model_str += ".pt"

    model.load_state_dict(torch.load(model_str))
    model.eval()

    #image_path = list_of_test_images[501]
    opencv_image = cv_image_read(image_path)
    inference_result = inference_image(opencv_image, transform_info, model, DEVICE)
    inference_result = inference_result.cpu().detach().numpy()
    print_result(inference_result, class_map)
    show_image(opencv_image)

테스트 함수는 추론을 위한 라벨 파일을 읽고 모델 파일을 로딩한 후, GPU 사용과 입력 이미지를 224X224로 변환을 지정해 이미지를 추론

In [ ]:
# 테스트 이미지로 테스트 메인 함수 실행 1
image_path = list_of_test_images[10]
testmain(image_path)

# 테스트 이미지로 테스트 메인 함수 실행 2
image_path = list_of_test_images[501]
testmain(image_path)

추론 결과를 라벨과 이미지를 화면에 표시